<a href="https://colab.research.google.com/github/gacerioni/redis-workshop-json-search-vs/blob/master/redis_workshop_vector_intro_pt_br_gabs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop - Redis como VectorDB - INTRO (TEM OUTROS!)

## Vector Searches & Large Language Models

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)


Bem-vind[ao]s ao Workshop! Vamos ter uma experiência hands-on sobre alguns temas centrais do Redis, bem além do Caching.


Para uma experiência premium, como a que eu quero que vocês tenham, recomendo MUITO utilizar o Redis Insight (App ou Web) pra apoiar na visualização dos dados.

https://redis.com/redis-enterprise/redis-insight/

---

Novamente, vamos direto ao ponto. Para pegar o fio da meada, passando pela introdução, veja este outro notebook [aqui](https://colab.research.google.com/github/gacerioni/redis-workshop-json-search-vs/blob/master/redis-workshop-vector-similarity-search.ipynb).

---

## Objetivos do Workshop

Este Notebook é uma pequena demonstração do Redis como um Vector DB. Depois, vamos ver uma implementação de RAG e Semantic/LLM Caching.


Espero que gostem! 🖖


## Conceito - Bancos de dados de vetores

Os dados são frequentemente não estruturados, o que significa que não são descritos por um esquema bem definido. Exemplos de dados não estruturados incluem trechos de texto, imagens, vídeos ou áudio. Uma abordagem para armazenar e pesquisar dados não estruturados é usar embeddings de vetores.

**O que são vetores?**\
Em aprendizado de máquina e IA, vetores são sequências de números que representam dados. Eles são as entradas e saídas dos modelos, encapsulando informações subjacentes em uma forma numérica. Vetores transformam dados não estruturados, como textos, imagens, vídeos e áudios, em um formato que os modelos de aprendizado de máquina podem processar.

**Por que eles são importantes?**\
Vetores capturam padrões complexos e significados semânticos inerentes aos dados, tornando-os ferramentas poderosas para uma variedade de aplicações. Eles permitem que modelos de aprendizado de máquina compreendam e manipulem dados não estruturados de forma mais eficaz.

**Melhorando a busca tradicional.**\
A busca tradicional por palavras-chave ou lexical depende de correspondências exatas de palavras ou frases, o que pode ser limitante. Em contraste, a busca vetorial, ou busca semântica, aproveita a rica informação capturada nos embeddings de vetores. Ao mapear dados em um espaço vetorial, itens semelhantes são posicionados próximos uns dos outros com base em seu significado. Essa abordagem permite resultados de busca mais precisos e significativos, pois considera o contexto e o conteúdo semântico da consulta, e não apenas as palavras exatas usadas.

# Passo 1 - Criar uma conta Free no Redis Cloud

Basta seguir o passo a passo [aqui](https://colab.research.google.com/github/gacerioni/redis-workshop-notebook-validator/blob/master/redis-workshop-setup-notebook-validator.ipynb)!

# Setup Rápido

## Instalaçao das libs do Python e redis-cli

In [ ]:
# Instale o Redis client e tambem o Hugging Face sentence transformers, pois vamos gerar os vetores aqui mesmo
!pip install -q redis sentence_transformers

# Instale tbm algumas libs pra gente brincar com LLM
# este comando baixa umas paradas de 700MB+, ok? Leva dois minutinhos.
!pip install -r https://raw.githubusercontent.com/gacerioni/redis-workshop-json-search-vs/master/deps/llm-movies/requirements.txt

# E instalar a CLI, via redis-tools, que inclui a famosa redis-cli
!apt-get update
!apt-get install -y redis-tools

## Deployment do Redis Stack - Um passo importante para esta demo em específico

Pessoal, esse Workshop de VectorSearch vai passar de 30MB.

---

**Ou seja: pra gente poder fazer esse e os futuros de Vector com LLM, vamos rodar o `redis-stack` aqui, locamente, neste notebook mesmo.**

---

Para isso, basta executar:

In [ ]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

### Conectando com o Redis server

In [ ]:
import os

# Coloque aqui os dados do seu DB do Redis Cloud
REDIS_HOST="localhost"
REDIS_PORT=6379
REDIS_PASSWORD=""

# Caso o SSL esteja ativo pro endpoint, adicione --tls
# Recomendo não misturar lé com cré aqui, visto que não vamos ter nenhuma informação sensível passando pelo fio.
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# Caso o SSL esteja ativo pro endpoint, use rediss:// como o URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"
INDEX_NAME = f"qna:idx"

# Teste a Redis connection
!redis-cli $REDIS_CONN PING

In [ ]:
# Testando via Python (redis-py)
import redis
redis = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD)
redis.ping()

## Importando e preparando as libs que iremos usar

Este primeiro bloco vai garantir que todas as dependências estejam prontas pra gente brincar com o ChatGPT usando o Redis como um cérebro extra e atualizado.

O bloco de variáveis ali é basicamente eu explicando algumas preferências minhas pro Vector Search, como quantas dimensões meus vetores possuem. 384, neste caso.

In [ ]:
import redis
import csv
import os
import numpy as np
from sentence_transformers import *
from redis.commands.search.query import Query
from redis.commands.search.field import TextField, TagField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
import openai
import tiktoken


redis = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD,
  decode_responses=True)

# Preferencias acerca do nosso Vector Search e como o Redis deve indexar os embeddings
VSS_INDEX_TYPE = "HNSW"
VSS_DATA_TYPE = "FLOAT32"
VSS_DISTANCE = "COSINE"
VSS_DIMENSION = 384
VSS_MINIMUM_SCORE = 2

MAX_MOVIES = 5000000


redis.ping()


# Iniciando os trabalhos

## Passo 1 - Carregando os filmes no Redis

Primeiro, vamos baixar o CSV do nosso GitHub, contendo um monte de filmes.\
Eu fiz um dump da base do **[IMDB](https://www.imdb.com/)**, caso estejam curiosos.






In [ ]:
# Baixando
!wget https://raw.githubusercontent.com/gacerioni/gabs-chatbot-llm-gpt-redis-vector-rag-demo/main/data/movies/imdb_movies.csv

In [ ]:
def load():
    with open("imdb_movies.csv", encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
        cnt = 0
        for row in csvReader:
            redis.json().set(f'moviebot:movie:{cnt}', '$', row)
            cnt = cnt + 1
            if (cnt > MAX_MOVIES):
                break
        print("Data was loaded into Redis!")

# Flush no DB, pra gente começar do 0
redis.flushdb()

# Carregar os dados no Redis
load()


# Contar as chaves no Redis, só pra ver se tudo foi carregado
key_count = redis.dbsize()
print(f"Total de chaves no Redis: {key_count}")

E ver alguns dados, só pra ter certeza que subiram para o Redis.

Notem que a gente não criou os vetores ainda, belê?

In [ ]:
redis.json().get("moviebot:movie:1")

## Passo 2 - Embedding - Hora de gerar vetores!

Finalmente, vamos ver nosso banco de vetores tomando forma.

Para isso, pegaremos alguns campos de informações não estruturadas de cada filme. Vamos concatenar dados interessantes como título, gênero, equipe, pontuação e sinopse.

Com essa string concatenada, geraremos vetores que representam cada filme de maneira embutida. Esses vetores serão armazenados no Redis, permitindo buscas e análises semânticas avançadas sobre os filmes.

In [ ]:
# esse comando pode levar 1-2 minutos, ok?
def create_embeddings():
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    for key in redis.scan_iter(match='moviebot:movie:*'):
        print(f"creating the embedding for {key}")
        result = redis.json().get(key, "$.names", "$.overview", "$.crew", "$.score", "$.genre")
        movie = f"movie title is: {result['$.names'][0]}\n"
        movie += f"movie genre is: {result['$.genre'][0]}\n"
        movie += f"movie crew is: {result['$.crew'][0]}\n"
        movie += f"movie score is: {result['$.score'][0]}\n"
        movie += f"movie overview is: {result['$.overview'][0]}\n"
        redis.json().set(key, "$.overview_embedding", model.encode(movie).astype(np.float32).tolist())

create_embeddings()

#### Checkpoint - vamos entender o que temos no Redis agora

Nós criamos o embedding usando outros atributos do filme, e guardamos esse "fingerprint" junto do próprio documento JSON que representa cada filme.

Olha só como ficou agora:

In [ ]:
import json

movie_with_embedding = redis.json().get("moviebot:movie:1")
pretty_json = json.dumps(movie_with_embedding, indent=4, ensure_ascii=False)
print(pretty_json)

## Passo 3 - Criando o índex para o JSON do Filme

Vamos criar o índex, sem segredo, pra gente fazer as queries depois.

In [ ]:
import time

def create_index():
    indexes = redis.execute_command("FT._LIST")
    if "movie_idx" not in indexes:
        index_def = IndexDefinition(prefix=["moviebot:movie:"], index_type=IndexType.JSON)
        schema = (TextField("$.crew", as_name="crew"),
                  TextField("$.overview", as_name="overview"),
                  TagField("$.genre", as_name="genre"),
                  TagField("$.names", as_name="names"),
                  VectorField("$.overview_embedding", VSS_INDEX_TYPE,
                              {"TYPE": VSS_DATA_TYPE, "DIM": VSS_DIMENSION, "DISTANCE_METRIC": VSS_DISTANCE},
                              as_name="embedding"))
        redis.ft('movie_idx').create_index(schema, definition=index_def)
        print("The index has been created")
    else:
        print("The index exists")

# Cria o index aqui mesmo
create_index()

E vamos testar o index por aqui mesmo também, fazendo uma Full Text Search bem preguiçosa

In [ ]:
# Testa aqui mesmo
time.sleep(2)

# Executar a busca, direto aqui mesmo
# Redis é binary safe, então esse abaixo, com ñ, precisa funfar de boa
search_result = redis.execute_command("FT.SEARCH", "movie_idx", "@crew:'Zoe Saldaña'")
print(search_result)

# Passo 4 - Criando as funções Python de apoio ao ChatGPT LLM

Vamos usar este pequeno bloco de código para configurar alguns detalhes para fazer funcionar toda a nossa integração.

---

### O que deve acontecer agora?

**Estas duas funções abaixo são para orquestrar a comunicação entre o usuário/cliente e o chatgpt.**

Isso por si só já é um RAG! Em detalhes, é isso que ocorre:



*   Pedimos para o usuário fazer uma pergunta em linguagem natural. Neste caso, buscando por filmes.
*   Transformamos a busca do cliente em um vetor usando o mesmo modelo utilizado para criar os embeddings.
*   Realizamos a busca no Redis usando o vetor. Um Vector Search por distância de cosseno, pois funciona bem pro nosso caso de uso.
*   Pegamos os filmes encontrados, e vamos usá-los como contexto pro ChatGPT ficar mais ligeiro sobre o assunto que estamos falando. É aqui que a gente começa a diminuir as temidas alucinações do GPT4!
*   Preparamos uma mensagem sistêmica para definir o tom do LLM e especificar o que esperamos dele. Enviamos em texto pro chatgpt, dizendo o que esperamos dele.
*   Geramos a interação, guardamos a resposta, e entregamos ao cliente.

In [ ]:
VSS_LLM_RAG_MODEL = "gpt-3.5-turbo-0613"
OPENAI_API_KEY = "nada"
VSS_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

def get_prompt(model, query):
    context = ""
    prompt = ""

    # Configura a query de busca vetorial no Redis
    q = Query("@embedding:[VECTOR_RANGE $radius $vec]=>{$YIELD_DISTANCE_AS: score}") \
        .sort_by("score", asc=True) \
        .return_fields("overview", "names", "score", "$.crew", "$.genre", "$.score") \
        .paging(0, 5) \
        .dialect(2)

    # Define os parâmetros da query
    query_params = {
        "radius": VSS_MINIMUM_SCORE,
        "vec": model.encode(query).astype(np.float32).tobytes()
    }

    # Executa a busca no Redis
    res = redis.ft("movie_idx").search(q, query_params)

    # Processa os resultados da busca
    if (res is not None) and len(res.docs):
        it = iter(res.docs[0:])
        for x in it:
            movie = f"movie title is: {x['names']}\n"
            movie += f"movie genre is: {x['$.genre']}\n"
            movie += f"movie crew is: {x['$.crew']}\n"
            movie += f"movie score is: {x['$.score']}\n"
            movie += f"movie overview is: {x['overview']}\n"
            context += movie + "\n"

    # Cria o prompt para o LLM
    if len(context) > 0:
        prompt = '''Use the provided information to answer the search query the user has sent. The information in the
        database provides three movies, choose the one or the ones that fit most. If you can't answer the user's
        question, say "Sorry, I am unable to answer the question, try to refine your question". Do not guess. You
        must deduce the answer exclusively from the information provided. The answer must be formatted in markdown or
        HTML. Do not make things up. Do not add personal opinions. Do not add any disclaimer.

            Search query:

            {}

            Information in the database:

            {}
            '''.format(query, context)

    return prompt

def getOpenAIGPT35(prompt):
    # Define a mensagem do sistema
    system_msg = ('You are a smart and knowledgeable AI assistant with expertise in all kinds of movies. You are a '
                  'very friendly and helpful AI. You are empowered to recommend movies based on the provided context. '
                  'Do NOT make anything up. Do NOT engage in topics that are not about movies.')

    # Define a codificação
    encoding = tiktoken.encoding_for_model(VSS_LLM_RAG_MODEL)

    try:
        openai.api_key = OPENAI_API_KEY
        response = openai.ChatCompletion.create(model=VSS_LLM_RAG_MODEL,
                                                stream=False,
                                                messages=[{"role": "system", "content": system_msg},
                                                          {"role": "user", "content": prompt}])
        return response["choices"][0]["message"]["content"]
    except openai.error.OpenAIError as e:
        # Trata erros
        if "context window is too large" in str(e):
            print("Error: Maximum context length exceeded. Please shorten your input.")
            return "Maximum context length exceeded"
        else:
            print("An unexpected error occurred:", e)
            return "An unexpected error occurred"

# Passo 5 - Hora de testar pra valer!

**Parabéns por ter chegado até aqui!**

Vamos fazer algo bem simples, pra fechar com chave de ouro.

Vamos criar um loop pra fazer essa ponte entre a nossa APP e o ChatGPT.\
Isso é literalmente o que vemos em produção, nos clientes.







In [ ]:
def render():
    model = SentenceTransformer(VSS_MODEL)
    # Reage à entrada do usuário
    while True:
        question = input("Me peça indicações sobre filmes:\n")  # Pede para o usuário fazer uma pergunta
        reply = f"Sua pergunta foi: {question}"
        prompt = get_prompt(model, question)  # Gera o prompt a partir da pergunta do usuário
        response = getOpenAIGPT35(prompt)  # Obtém a resposta do LLM
        print(response)  # Exibe a resposta
        print("--------------------------------")

# Chama o loop por tempo indeterminado
render()